# Massachusetts School Data Investigation
The following is our investigation into the 2018-19 education data for Massachusetts. All of the data, except the NERDS dataset, has been sourced by the DOE website [here](https://profiles.doe.mass.edu/state_report/).
### Alex Craig & Aaron Xie

# Step 0: Loading, Clean, and Merge Data

## 0.1 Import Libraries

In [1034]:
import pandas as pd
import re

## 0.2 Initial Data Loading

In [1035]:
nerds_df = pd.read_excel("../data/MA_1819_final_January_28th_22.xlsx")
nerds_df.head()

,state,year,distid_stateassigned,schoolid_stateassigned,distname,schoolname,ncesdistid_admin,ncesdistid_geo,census_id,ncesid,...,pp_PD_cent_fed_raw_MA,pp_instrumaterial_cent_fed_raw_MA,pp_guidepsych_cent_fed_raw_MA,countenroll,noaudit,nositecen,adfinvars,distflex,flag_nerds,flag_f33
0,MA,2018-2019,445,105,Abby Kelley Foster Charter Public School,Abby Kelley Foster Charter Public School,2500051,2500051,NaN,250005101247,...,0,0,0,0,0,0,1,0,0,0
1,MA,2018-2019,1,1,Abington,Abington Early Education Program,2501650,2501650,2.230120e+13,250165002727,...,0,0,0,0,0,0,1,0,0,0
2,MA,2018-2019,1,505,Abington,Abington High,2501650,2501650,2.230120e+13,250165000001,...,0,0,0,0,0,0,1,0,0,0
3,MA,2018-2019,1,405,Abington,Abington Middle School,2501650,2501650,2.230120e+13,250165000002,...,0,0,0,0,0,0,1,0,0,0
4,MA,2018-2019,1,20,Abington,Beaver Brook Elementary,2501650,2501650,2.230120e+13,250165000434,...,0,0,0,0,0,0,1,0,0,0


In [1036]:
mcas_elem_df = pd.read_excel("../data/mcas_elem_results_2019.xlsx")
mcas_elem_df.head()

,School Name,School Code,Subject,M+E #,M+E %,E #,E %,M #,M %,PM #,PM %,NM #,NM %,No. of Students Included,Avg. Scaled Score,SGP,Included In SGP,Ach. PCTL
0,Abby Kelley Foster Charter Public (District) -...,4450105,ELA,346,49,37,5,309,44,303,43,50,7,699,499.4,51.4,578,38.0
1,Abby Kelley Foster Charter Public (District) -...,4450105,MATH,282,40,23,3,259,37,357,51,60,9,699,495.9,51.3,579,35.0
2,Abby Kelley Foster Charter Public (District) -...,4450105,SCI,80,34,4,2,76,32,135,57,21,9,236,492.6,NaN,NaN,27.0
3,Abington - Abington Middle School,10405,ELA,351,54,51,8,300,46,257,39,46,7,654,501.8,46.6,634,47.0
4,Abington - Abington Middle School,10405,MATH,294,45,29,4,265,40,308,47,53,8,655,497.4,44.2,634,41.0


In [1037]:
mcas_hs_df = pd.read_excel("../data/mcas_hs_results_2019.xlsx")
mcas_hs_df.head()

,School Name,School Code,Subject,M+E #,M+E %,E #,E %,M #,M %,PM #,PM %,NM #,NM %,No. of Students Included,Avg. Scaled Score,SGP,Included In SGP,Ach. PCTL
0,Abby Kelley Foster Charter Public (District) -...,4450105,ELA,62,65,11,12,51,54,33,35,0,0,95,507.5,55.0,94,53.0
1,Abby Kelley Foster Charter Public (District) -...,4450105,MATH,49,52,11,12,38,40,44,46,2,2,95,505.2,70.1,93,55.0
2,Abington - Abington High,10505,ELA,93,64,12,8,81,56,48,33,4,3,145,507.5,46.4,138,53.0
3,Abington - Abington High,10505,MATH,84,58,16,11,68,47,55,38,6,4,145,505.6,48.6,138,56.0
4,Academy Of the Pacific Rim Charter Public (Dis...,4120530,ELA,41,64,5,8,36,56,22,34,1,2,64,506.7,59.6,59,48.0


In [1038]:
enrollment_demographics_df = pd.read_excel("../data/enrollment_demographics_2019.xlsx")
enrollment_demographics_df.head()

,School Name,School Code,African American,Asian,Hispanic,White,Native American,"Native Hawaiian, Pacific Islander","Multi-Race, Non-Hispanic",Males,Females
0,Abby Kelley Foster Charter Public (District) -...,4450105,51.3,3.4,22.7,19.7,0.2,0.1,2.5,47.5,52.5
1,Abington - Abington Early Education Program,10001,10.0,1.1,10.0,78.9,0.0,0.0,0.0,50.0,50.0
2,Abington - Abington High,10505,3.9,2.4,7.9,85.0,0.0,0.2,0.7,49.0,51.0
3,Abington - Abington Middle School,10405,2.1,1.8,7.1,87.4,0.4,0.0,1.2,52.4,47.6
4,Abington - Beaver Brook Elementary,10020,4.8,2.4,7.3,84.0,1.1,0.0,0.4,50.5,49.5


In [1039]:
grad_rates_df = pd.read_excel("../data/graduation_rates_2019.xlsx")
grad_rates_df.head()

,School Name,School Code,# in Cohort,% Graduated,% Still in School,% Non-Grad Completers,% H.S. Equiv.,% Dropped Out,% Permanently Excluded
0,Abby Kelley Foster Charter Public (District) -...,4450105,84,97.6,0.0,0.0,0.0,2.4,0.0
1,Abington - Abington High,10505,115,91.3,1.7,0.9,0.0,6.1,0.0
2,Academy Of the Pacific Rim Charter Public (Dis...,4120530,40,95.0,5.0,0.0,0.0,0.0,0.0
3,Acton-Boxborough - Acton-Boxborough Regional High,6000505,493,97.8,1.2,0.0,0.2,0.8,0.0
4,Adams-Cheshire - Hoosac Valley High School,6030505,88,89.8,2.3,1.1,2.3,4.5,0.0


In [1040]:
socio_eco_df = pd.read_excel("../data/selectedpopulations_2019.xlsx")
socio_eco_df.head()

,School Name,School Code,First Language Not English #,First Language Not English %,English Language Learner #,English Language Learner %,Students With Disabilities #,Students With Disabilities %,Low Income #,Low Income %,Free Lunch #,Free Lunch %,Reduced Lunch #,Reduced Lunch %,High Needs #,High Needs #.1,Economically Disadvantaged #,Economically Disadvantaged %
0,Abby Kelley Foster Charter Public (District) -...,4450105,910,63.7,141,9.9,171,12.0,,,,,,,803,56.2,538,37.7
1,Abington - Abington Early Education Program,10001,9,10.0,2,2.2,34,37.8,,,,,,,46,51.1,22,24.4
2,Abington - Abington High,10505,37,6.8,10,1.8,69,12.7,,,,,,,161,29.5,105,19.3
3,Abington - Abington Middle School,10405,67,10.0,25,3.7,91,13.5,,,,,,,243,36.2,164,24.4
4,Abington - Beaver Brook Elementary,10020,68,14.9,44,9.7,65,14.3,,,,,,,158,34.7,89,19.6


In [1041]:
staff_demographics_df = pd.read_excel("../data/staff_demographics_2019.xlsx")
staff_demographics_df.head()

,District/School Name,District/School Code,African American (%),Asian (%),Hispanic (%),White (%),Native American (%),"Native Hawaiian, Pacific Islander (%)","Multi-Race,Non-Hispanic (%)",Females (%),Males (%),FTE Count
0,Abby Kelley Foster Charter Public (District) -...,4450105,3.7,0.9,1.8,92.4,0.0,0.0,1.2,79.2,20.8,163.0
1,Abington - Abington Early Education Program,10001,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0,0.0,16.0
2,Abington - Abington High,10505,0.0,1.1,1.6,95.7,0.0,0.0,1.6,72.5,27.5,62.4
3,Abington - Abington Middle School,10405,2.7,0.0,4.8,92.5,0.0,0.0,0.0,77.3,22.7,73.7
4,Abington - Beaver Brook Elementary,10020,0.0,0.0,1.7,98.3,0.0,0.0,0.0,99.3,0.7,57.7


In [1042]:
student_to_teacher_df = pd.read_excel("../data/teacherdata_2019.xlsx")
student_to_teacher_df.head()

,School Name,School Code,Total # of Teachers (FTE),% of Teachers Licensed,Student / Teacher Ratio,Percent of Experienced Teachers,Percent of Teachers without Waiver or Provisional License,Percent Teaching In-Field
0,Abby Kelley Foster Charter Public (District) -...,4450105,116.4,94.5,12.3 to 1,76.4,87.1,85.1
1,Abington - Abington Early Education Program,10001,4.0,100.0,22.5 to 1,100.0,100.0,100.0
2,Abington - Abington High,10505,37.9,100.0,14.4 to 1,92.6,88.9,83.3
3,Abington - Abington Middle School,10405,38.2,100.0,17.6 to 1,87.2,95.0,93.0
4,Abington - Beaver Brook Elementary,10020,25.7,100.0,17.7 to 1,84.5,96.1,100.0


In [1043]:
teacher_salary_df = pd.read_excel("../data/teachersalaries_2019.xlsx")
teacher_salary_df.head()

,District Name,District Code,Salary Totals,Average Salary,FTE Count
0,Abington,10000,"$11,469,788","$93,861",122.2
1,Acton-Boxborough,6000000,"$32,945,978","$86,677",380.1
2,Acushnet,30000,"$5,217,686","$69,108",75.5
3,Adams-Cheshire,6030000,"$6,920,007","$84,212",82.2
4,Agawam,50000,"$25,584,999","$79,828",320.5


## 0.3 Cleaning Data

### 0.3.1 Cleaning the NERDS Data
Lets clean the Nerds dataframe by only keeping the columns that we want, as it currently has many columns, most of which we don't need.

Columns for future Merges:
- `distname`: District Name
- `schoolname`: School Name

Columns for Spending Per Pupil:
- `pp_stloc`: State and Local Spending Per Pupil
- `pp_fed`: Federal Spending Per Pupil
- `pp_total`: Total Spending Per Pupil

Columns for Total Spending Per Pupil:
- `schoolstloc`: State and Local Spending
- `schoolfed`: Federal Spending
- `schooltot`: Total Spending

Columns for Enrollment:
- `ncesenroll`: Enrollment
- `gradespan`: Grades Spanned

Columns for Flagging:
- `flag_nerds`: Flag for suspicious data on school
- `flag_f33`: Flag for suspicious data on district

For this investigation, since we are not comparing across states, we are opting to use to `_raw` instead of the `_norm_nerds` columns which contain values which have been adjusted for state differences.

The information about these columns was found in the [NERDS Documentation](https://georgetown.app.box.com/s/nyt1ckphiblaj4tyuoigtri8bn38kk85/file/1080532555684).

In [1044]:
nerds_df_clean = nerds_df[['distname', 'schoolname', 'flag_nerds', 'flag_f33', 'ncesenroll', 'gradespan', 'pp_stloc_raw_MA', 'pp_fed_raw_MA', 'pp_total_raw_MA', 'schoolstloc_raw_MA', 'schoolfed_raw_MA', 'schooltot_raw_MA']]

# Let's also make district and school names lowercase for easier merging
nerds_df_clean["distname_lower"] = nerds_df_clean["distname"].str.lower()
nerds_df_clean["schoolname_lower"] = nerds_df_clean["schoolname"].str.lower()

nerds_df_clean.head()

/var/folders/1y/tx5xv7mx325cz5z1q7xc35sm0000gn/T/ipykernel_28852/3621529377.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nerds_df_clean["distname_lower"] = nerds_df_clean["distname"].str.lower()
/var/folders/1y/tx5xv7mx325cz5z1q7xc35sm0000gn/T/ipykernel_28852/3621529377.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nerds_df_clean["schoolname_lower"] = nerds_df_clean["schoolname"].str.lower()


,distname,schoolname,flag_nerds,flag_f33,ncesenroll,gradespan,pp_stloc_raw_MA,pp_fed_raw_MA,pp_total_raw_MA,schoolstloc_raw_MA,schoolfed_raw_MA,schooltot_raw_MA,distname_lower,schoolname_lower
0,Abby Kelley Foster Charter Public School,Abby Kelley Foster Charter Public School,0,0,1428.0,KG-12,14354.612514,1006.982538,15361.595053,20461351.77,1435373.05,21896724.82,abby kelley foster charter public school,abby kelley foster charter public school
1,Abington,Abington Early Education Program,0,0,90.0,PK-PK,14028.783797,961.863052,14990.646848,1444029.478804,99007.770134,1543037.248938,abington,abington early education program
2,Abington,Abington High,0,0,545.0,09-12,14880.475672,20.812074,14901.287746,8052652.079188,11262.569511,8063914.648699,abington,abington high
3,Abington,Abington Middle School,0,0,672.0,05-08,13057.825084,83.417661,13141.242745,8839857.407697,56471.903107,8896329.310804,abington,abington middle school
4,Abington,Beaver Brook Elementary,0,0,455.0,KG-02,13134.592802,1105.059138,14239.65194,5944497.792223,500131.348068,6444629.140291,abington,beaver brook elementary


Let's make lower case columns for `distname` and `schoolname` for easier merging later.

In [1045]:
nerds_df_clean["distname_lower"] = nerds_df_clean["distname"].str.lower()
nerds_df_clean["schoolname_lower"] = nerds_df_clean["schoolname"].str.lower()

nerds_df_clean.head()

/var/folders/1y/tx5xv7mx325cz5z1q7xc35sm0000gn/T/ipykernel_28852/2373075718.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nerds_df_clean["distname_lower"] = nerds_df_clean["distname"].str.lower()
/var/folders/1y/tx5xv7mx325cz5z1q7xc35sm0000gn/T/ipykernel_28852/2373075718.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nerds_df_clean["schoolname_lower"] = nerds_df_clean["schoolname"].str.lower()


,distname,schoolname,flag_nerds,flag_f33,ncesenroll,gradespan,pp_stloc_raw_MA,pp_fed_raw_MA,pp_total_raw_MA,schoolstloc_raw_MA,schoolfed_raw_MA,schooltot_raw_MA,distname_lower,schoolname_lower
0,Abby Kelley Foster Charter Public School,Abby Kelley Foster Charter Public School,0,0,1428.0,KG-12,14354.612514,1006.982538,15361.595053,20461351.77,1435373.05,21896724.82,abby kelley foster charter public school,abby kelley foster charter public school
1,Abington,Abington Early Education Program,0,0,90.0,PK-PK,14028.783797,961.863052,14990.646848,1444029.478804,99007.770134,1543037.248938,abington,abington early education program
2,Abington,Abington High,0,0,545.0,09-12,14880.475672,20.812074,14901.287746,8052652.079188,11262.569511,8063914.648699,abington,abington high
3,Abington,Abington Middle School,0,0,672.0,05-08,13057.825084,83.417661,13141.242745,8839857.407697,56471.903107,8896329.310804,abington,abington middle school
4,Abington,Beaver Brook Elementary,0,0,455.0,KG-02,13134.592802,1105.059138,14239.65194,5944497.792223,500131.348068,6444629.140291,abington,beaver brook elementary


### 0.3.2 Cleaning the MCAS Data
MCAS is the standardized test which Massachusetts uses for all grade levels through high school. Let's convert the dataframe to a wide format so that each District is a single row, and each column is a different subject. We will also keep the `School Name` and `School Code` columns for future merges. Keep in mind we have separate dataframes for elementary and high school.

In [1046]:
mcas_elem_df_clean = pd.pivot(mcas_elem_df, index=['School Name', 'School Code'], columns='Subject', values='Avg. Scaled Score').reset_index()
mcas_elem_df_clean.head()

Subject,School Name,School Code,ELA,MATH,SCI
0,Abby Kelley Foster Charter Public (District) -...,4450105,499.4,495.9,492.6
1,Abington - Abington Middle School,10405,501.8,497.4,496.8
2,Abington - Woodsdale Elementary School,10015,508.5,505.1,NaN
3,Academy Of the Pacific Rim Charter Public (Dis...,4120530,496.5,491.4,491.6
4,Acton-Boxborough - Blanchard Memorial School,6000005,514.7,513.8,521.2


In [1047]:
mcas_hs_df_clean = pd.pivot(mcas_hs_df, index=['School Name', 'School Code'], columns='Subject', values='Avg. Scaled Score').reset_index()
mcas_hs_df_clean.head()

Subject,School Name,School Code,ELA,MATH
0,Abby Kelley Foster Charter Public (District) -...,4450105,507.5,505.2
1,Abington - Abington High,10505,507.5,505.6
2,Academy Of the Pacific Rim Charter Public (Dis...,4120530,506.7,498.7
3,Acton-Boxborough - Acton-Boxborough Regional High,6000505,521.6,528.8
4,Advanced Math and Science Academy Charter (Dis...,4300305,525.9,533.1


We see that `School Name` contains both school and district names. Let's split them up, convert to lower case, and also remove the "(District)" from the end of the district name. Also, we will make this process a function to use for the cleaning of other Massachusetts data. 

In [1048]:
split_names_elem = [name.split(" - ") for name in mcas_elem_df_clean["School Name"]]
print([name for name in split_names_elem if len(name) != 2])

[['Community Day Charter Public School', 'Gateway (District)', 'Community Day Charter Public School', 'Gateway'], ['Community Day Charter Public School', 'R. Kingman Webster (District)', 'Community Day Charter Public School', 'R. Kingman Webster']]


In [1049]:
split_names_hs = [name.split(" - ") for name in mcas_hs_df_clean["School Name"]]
print([name for name in split_names_hs if len(name) != 2])

[]


We see a few schools with names that are not in the format 'District - School'. We will have to write some logic to handle these cases. We can see that there are two cases, and in both of them school comes first, district comes second, and the length of their list is greater than 2.

In [1050]:
def clean_school_name(dataframe, column_name):
    split_names = [name.split(" - ") for name in dataframe[column_name]]
    
    dist_names_lower = [name[0].lower() if len(name) == 2 else name[1].lower() for name in split_names]
    school_names_lower = [name[1].lower() if len(name) == 2 else name[0].lower() for name in split_names]

    regex = r"\s*\(\s*district\s*\)\s*"
    dist_names_lower_clean = [re.sub(regex, "", name) if name is not None else None for name in dist_names_lower]

    dataframe["distname_lower"] = dist_names_lower_clean
    dataframe["schoolname_lower"] = school_names_lower

In [1051]:
clean_school_name(mcas_elem_df_clean, "School Name")
clean_school_name(mcas_hs_df_clean, "School Name")

# Let's check the edge cases we found
mcas_elem_df_clean[mcas_elem_df_clean["distname_lower"] == "gateway"]
mcas_elem_df_clean[mcas_elem_df_clean["distname_lower"] == "r. kingman webster"]

Subject,School Name,School Code,ELA,MATH,SCI,distname_lower,schoolname_lower
328,Community Day Charter Public School - R. Kingm...,4310205,514.6,512.9,502.9,r. kingman webster,community day charter public school


Next, let's make a `avg_score` column which is the average of the 3 MCAS scores. There are a few NaN scores, however these should still work with the `mean` function.

In [1052]:
avg_score_elem = mcas_elem_df_clean[['ELA', 'MATH', 'SCI']].mean(axis=1)
mcas_elem_df_clean["avg_score"] = avg_score_elem

mcas_elem_df_clean.head()

Subject,School Name,School Code,ELA,MATH,SCI,distname_lower,schoolname_lower,avg_score
0,Abby Kelley Foster Charter Public (District) -...,4450105,499.4,495.9,492.6,abby kelley foster charter public,abby kelley foster charter public school,495.966667
1,Abington - Abington Middle School,10405,501.8,497.4,496.8,abington,abington middle school,498.666667
2,Abington - Woodsdale Elementary School,10015,508.5,505.1,NaN,abington,woodsdale elementary school,506.800000
3,Academy Of the Pacific Rim Charter Public (Dis...,4120530,496.5,491.4,491.6,academy of the pacific rim charter public,academy of the pacific rim charter public school,493.166667
4,Acton-Boxborough - Blanchard Memorial School,6000005,514.7,513.8,521.2,acton-boxborough,blanchard memorial school,516.566667


In [1053]:
avg_score_hs = mcas_hs_df_clean[['ELA', 'MATH']].mean(axis=1)
mcas_hs_df_clean["avg_score"] = avg_score_hs

mcas_hs_df_clean.head()

Subject,School Name,School Code,ELA,MATH,distname_lower,schoolname_lower,avg_score
0,Abby Kelley Foster Charter Public (District) -...,4450105,507.5,505.2,abby kelley foster charter public,abby kelley foster charter public school,506.35
1,Abington - Abington High,10505,507.5,505.6,abington,abington high,506.55
2,Academy Of the Pacific Rim Charter Public (Dis...,4120530,506.7,498.7,academy of the pacific rim charter public,academy of the pacific rim charter public school,502.70
3,Acton-Boxborough - Acton-Boxborough Regional High,6000505,521.6,528.8,acton-boxborough,acton-boxborough regional high,525.20
4,Advanced Math and Science Academy Charter (Dis...,4300305,525.9,533.1,advanced math and science academy charter,advanced math and science academy charter school,529.50


### 0.3.3 Cleaning the Enrollment Demographics Data
The `enrollment_demographics_df` comes from the same source as the `mcas_df`, which is Massachusett's Department of Education, so it is formatted in the same way and we just need to clean the `School Name` column.

In [1054]:
enrollment_demographics_df_clean = enrollment_demographics_df.copy()
clean_school_name(enrollment_demographics_df_clean, "School Name")

enrollment_demographics_df_clean.head()

,School Name,School Code,African American,Asian,Hispanic,White,Native American,"Native Hawaiian, Pacific Islander","Multi-Race, Non-Hispanic",Males,Females,distname_lower,schoolname_lower
0,Abby Kelley Foster Charter Public (District) -...,4450105,51.3,3.4,22.7,19.7,0.2,0.1,2.5,47.5,52.5,abby kelley foster charter public,abby kelley foster charter public school
1,Abington - Abington Early Education Program,10001,10.0,1.1,10.0,78.9,0.0,0.0,0.0,50.0,50.0,abington,abington early education program
2,Abington - Abington High,10505,3.9,2.4,7.9,85.0,0.0,0.2,0.7,49.0,51.0,abington,abington high
3,Abington - Abington Middle School,10405,2.1,1.8,7.1,87.4,0.4,0.0,1.2,52.4,47.6,abington,abington middle school
4,Abington - Beaver Brook Elementary,10020,4.8,2.4,7.3,84.0,1.1,0.0,0.4,50.5,49.5,abington,beaver brook elementary


### 0.3.4 Cleaning the Graduation Rates Data
The `grad_rates_df` also comes from Massachusett's Department of Education, so we just need to clean the `School Name` column in the same way. Keep in mind only high schools are included in this dataset.

In [1055]:
grad_rates_df_clean = grad_rates_df.copy()
clean_school_name(grad_rates_df_clean, "School Name")

grad_rates_df_clean.head()

,School Name,School Code,# in Cohort,% Graduated,% Still in School,% Non-Grad Completers,% H.S. Equiv.,% Dropped Out,% Permanently Excluded,distname_lower,schoolname_lower
0,Abby Kelley Foster Charter Public (District) -...,4450105,84,97.6,0.0,0.0,0.0,2.4,0.0,abby kelley foster charter public,abby kelley foster charter public school
1,Abington - Abington High,10505,115,91.3,1.7,0.9,0.0,6.1,0.0,abington,abington high
2,Academy Of the Pacific Rim Charter Public (Dis...,4120530,40,95.0,5.0,0.0,0.0,0.0,0.0,academy of the pacific rim charter public,academy of the pacific rim charter public school
3,Acton-Boxborough - Acton-Boxborough Regional High,6000505,493,97.8,1.2,0.0,0.2,0.8,0.0,acton-boxborough,acton-boxborough regional high
4,Adams-Cheshire - Hoosac Valley High School,6030505,88,89.8,2.3,1.1,2.3,4.5,0.0,adams-cheshire,hoosac valley high school


### 0.3.5 Cleaning the Socio-Economic Data
The `socio_eco_df` also comes from Massachusett's Department of Education, so we just need to clean the `School Name` column in the same way.

In [1056]:
socio_eco_df_clean = socio_eco_df.copy()
clean_school_name(socio_eco_df_clean, "School Name")

socio_eco_df_clean.head()

,School Name,School Code,First Language Not English #,First Language Not English %,English Language Learner #,English Language Learner %,Students With Disabilities #,Students With Disabilities %,Low Income #,Low Income %,Free Lunch #,Free Lunch %,Reduced Lunch #,Reduced Lunch %,High Needs #,High Needs #.1,Economically Disadvantaged #,Economically Disadvantaged %,distname_lower,schoolname_lower
0,Abby Kelley Foster Charter Public (District) -...,4450105,910,63.7,141,9.9,171,12.0,,,,,,,803,56.2,538,37.7,abby kelley foster charter public,abby kelley foster charter public school
1,Abington - Abington Early Education Program,10001,9,10.0,2,2.2,34,37.8,,,,,,,46,51.1,22,24.4,abington,abington early education program
2,Abington - Abington High,10505,37,6.8,10,1.8,69,12.7,,,,,,,161,29.5,105,19.3,abington,abington high
3,Abington - Abington Middle School,10405,67,10.0,25,3.7,91,13.5,,,,,,,243,36.2,164,24.4,abington,abington middle school
4,Abington - Beaver Brook Elementary,10020,68,14.9,44,9.7,65,14.3,,,,,,,158,34.7,89,19.6,abington,beaver brook elementary


### 0.3.6 Cleaning the Staff Demographics Data
The `staff_demographic_df` also comes from Massachusett's Department of Education, so we just need to clean the `School Name` column in the same way.

In [1057]:
staff_demographics_df_clean = staff_demographics_df.copy()
clean_school_name(staff_demographics_df_clean, "District/School Name")
staff_demographics_df_clean.rename(columns={"District/School Name": "School Name", "District/School Code": "School Code"}, inplace=True)

staff_demographics_df_clean.head()

,School Name,School Code,African American (%),Asian (%),Hispanic (%),White (%),Native American (%),"Native Hawaiian, Pacific Islander (%)","Multi-Race,Non-Hispanic (%)",Females (%),Males (%),FTE Count,distname_lower,schoolname_lower
0,Abby Kelley Foster Charter Public (District) -...,4450105,3.7,0.9,1.8,92.4,0.0,0.0,1.2,79.2,20.8,163.0,abby kelley foster charter public,abby kelley foster charter public school
1,Abington - Abington Early Education Program,10001,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0,0.0,16.0,abington,abington early education program
2,Abington - Abington High,10505,0.0,1.1,1.6,95.7,0.0,0.0,1.6,72.5,27.5,62.4,abington,abington high
3,Abington - Abington Middle School,10405,2.7,0.0,4.8,92.5,0.0,0.0,0.0,77.3,22.7,73.7,abington,abington middle school
4,Abington - Beaver Brook Elementary,10020,0.0,0.0,1.7,98.3,0.0,0.0,0.0,99.3,0.7,57.7,abington,beaver brook elementary


### 0.3.7 Cleaning the Student to Teacher Data
The `student_to_teacher_df` also comes from Massachusett's Department of Education, so we just need to clean the `School Name` column in the same way. However, we are only interested in the `Student / Teacher Ratio` column, so we will only keep and clean that.

In [1058]:
student_to_teacher_df_clean = student_to_teacher_df[["School Name", "School Code", "Student / Teacher Ratio"]].copy()
clean_school_name(student_to_teacher_df_clean, "School Name")
student_to_teacher_df_clean.rename(columns={"Student / Teacher Ratio": "student_teacher_ratio"}, inplace=True)

split_ratio = [str(ratio).split(" to ") for ratio in student_to_teacher_df_clean["student_teacher_ratio"]]
student_to_teacher_df_clean["student_teacher_ratio"] = [float(ratio[0]) for ratio in split_ratio]

student_to_teacher_df_clean[student_to_teacher_df_clean["student_teacher_ratio"].isna()]

,School Name,School Code,student_teacher_ratio,distname_lower,schoolname_lower
276,Boston - West Roxbury Academy,350658,NaN,boston,west roxbury academy
1482,Springfield - Gateway to College at Holyoke Co...,2810575,NaN,springfield,gateway to college at holyoke community college
1483,Springfield - Gateway to College at Springfiel...,2810580,NaN,springfield,gateway to college at springfield technical co...
1607,Uxbridge - Gateway to College,3040515,NaN,uxbridge,gateway to college


We see there are a few schools with `NaN` as the value to student_to_teacher_ratio. We will replace these with the mean of the column and flag them.

In [1059]:
student_to_teacher_df_clean["nan_flag"] = student_to_teacher_df_clean["student_teacher_ratio"].isna()
avg_student_teacher_ratio = student_to_teacher_df_clean["student_teacher_ratio"].mean()

student_to_teacher_df_clean["student_teacher_ratio"] = student_to_teacher_df_clean["student_teacher_ratio"].fillna(avg_student_teacher_ratio)

student_to_teacher_df_clean[student_to_teacher_df_clean["nan_flag"]]

,School Name,School Code,student_teacher_ratio,distname_lower,schoolname_lower,nan_flag
276,Boston - West Roxbury Academy,350658,13.646363,boston,west roxbury academy,True
1482,Springfield - Gateway to College at Holyoke Co...,2810575,13.646363,springfield,gateway to college at holyoke community college,True
1483,Springfield - Gateway to College at Springfiel...,2810580,13.646363,springfield,gateway to college at springfield technical co...,True
1607,Uxbridge - Gateway to College,3040515,13.646363,uxbridge,gateway to college,True


### 0.3.8 Cleaning the Teacher Salaries Data
Teacher salaries are only available by district, so we just need to convert the `District Name` column to lower case.

In [1060]:
teacher_salary_df_clean = teacher_salary_df.copy()
teacher_salary_df_clean["distname_lower"] = teacher_salary_df_clean["District Name"].str.lower()

teacher_salary_df_clean.head()

,District Name,District Code,Salary Totals,Average Salary,FTE Count,distname_lower
0,Abington,10000,"$11,469,788","$93,861",122.2,abington
1,Acton-Boxborough,6000000,"$32,945,978","$86,677",380.1,acton-boxborough
2,Acushnet,30000,"$5,217,686","$69,108",75.5,acushnet
3,Adams-Cheshire,6030000,"$6,920,007","$84,212",82.2,adams-cheshire
4,Agawam,50000,"$25,584,999","$79,828",320.5,agawam


## 0.4 Merging Data
Now we want to merge all of the disparate dataframes into one master dataframe. We can do so by merging on the `schoolname_lower` and `distname_lower` columns. Let's initialize the master dataframe with the `nerds_df`.

In [1061]:
master_df = nerds_df_clean.copy()

### 0.4.1 Merging the MCAS Data
We will merge the elementary and high school MCAS dataframes together. We will also make a new `avg_score` column with the average of high school and elementary scores for schools that have both.

In [1062]:
mcas_merged = pd.merge(mcas_elem_df_clean, mcas_hs_df_clean, how="outer", on=["distname_lower", "schoolname_lower", "School Code", "School Name"], suffixes=("_elem", "_hs"))
mcas_merged["avg_score"] = mcas_merged[["avg_score_elem", "avg_score_hs"]].mean(axis=1)

print("Looks like we have one school with no MCAS scores. Let's just drop it.")
mcas_merged[mcas_merged["avg_score"].isna()]

Looks like we have one school with no MCAS scores. Let's just drop it.


Subject,School Name,School Code,ELA_elem,MATH_elem,SCI,distname_lower,schoolname_lower,avg_score_elem,ELA_hs,MATH_hs,avg_score_hs,avg_score
636,Lowell - Dr. Janice Adie Day School,1600605,NaN,NaN,NaN,lowell,dr. janice adie day school,NaN,NaN,NaN,NaN,NaN


In [1063]:
mcas_merged = mcas_merged[~mcas_merged["avg_score"].isna()]

Now let's merge mcas_df with the master dataframe. We will use an inner here because we want to only keep schools that have MCAS data.

In [1064]:
master_df = pd.merge(master_df, mcas_merged, how="inner", on=["distname_lower", "schoolname_lower"], suffixes=("_nerds", "_mcas"))
master_df.shape

(1460, 24)

### 0.4.2 Merging the Enrollment Demographics Data

In [1065]:
master_df = pd.merge(master_df, enrollment_demographics_df_clean, how="inner", on=["distname_lower", "schoolname_lower", "School Code", "School Name"], suffixes=("", "_enroll"))
print("We only lose 7 schools with and inner join, so we will merge with an inner join.")
master_df.shape

We only lose 7 schools with and inner join, so we will merge with an inner join.


(1453, 33)

### 0.4.3 Merging the Graduation Rates Data
Graduation rates only apply to high schools, so we will use a left join here.

In [1066]:
master_df = pd.merge(master_df, grad_rates_df_clean, how="left", on=["distname_lower", "schoolname_lower", "School Code", "School Name"], suffixes=("", "_grad"))
master_df.shape

(1453, 40)

### 0.4.4 Merging the Socio-Economic Data

In [1067]:
master_df = pd.merge(master_df, socio_eco_df_clean, how="inner", on=["distname_lower", "schoolname_lower", "School Code", "School Name"], suffixes=("", "_socioeco"))
print("We don't lose any school with an inner join.")
master_df.shape

We don't lose any school with an inner join.


(1453, 56)

### 0.4.5 Merging the Staff Demographics Data

In [1068]:
master_df = pd.merge(master_df, staff_demographics_df_clean, how="inner", on=["distname_lower", "schoolname_lower", "School Code", "School Name"], suffixes=("", "_staff"))
print("Again, we don't lose any school with an inner join.")
master_df.shape

Again, we don't lose any school with an inner join.


(1453, 66)

### 0.4.6 Merging the Student to Teacher Data

In [1069]:
master_df = pd.merge(master_df, student_to_teacher_df_clean, how="inner", on=["distname_lower", "schoolname_lower", "School Code", "School Name"], suffixes=("", "_student_teacher"))
master_df.shape

(1453, 68)

### 0.4.7 Merging the Teacher Salaries Data
Teacher salaries are only available by district, so we will use a left join here.

In [1070]:
master_df = pd.merge(master_df, teacher_salary_df_clean, how="left", on=["distname_lower"], suffixes=("", "_teacher_salary"))
master_df.shape

(1453, 73)

### 0.4.8 Final Look at Master Dataframe

In [1071]:
master_df

,distname,schoolname,flag_nerds,flag_f33,ncesenroll,gradespan,pp_stloc_raw_MA,pp_fed_raw_MA,pp_total_raw_MA,schoolstloc_raw_MA,...,Females (%),Males (%),FTE Count,student_teacher_ratio,nan_flag,District Name,District Code,Salary Totals,Average Salary,FTE Count_teacher_salary
0,Abington,Abington High,0,0,545.0,09-12,14880.475672,20.812074,14901.287746,8052652.079188,...,72.5,27.5,62.4,14.4,False,Abington,10000.0,"$11,469,788","$93,861",122.2
1,Abington,Abington Middle School,0,0,672.0,05-08,13057.825084,83.417661,13141.242745,8839857.407697,...,77.3,22.7,73.7,17.6,False,Abington,10000.0,"$11,469,788","$93,861",122.2
2,Abington,Woodsdale Elementary School,0,0,294.0,03-04,13201.700857,499.583587,13701.284443,3841768.292088,...,88.8,11.2,39.4,18.0,False,Abington,10000.0,"$11,469,788","$93,861",122.2
3,Acton-Boxborough,Acton-Boxborough Regional High,0,0,1837.0,09-12,14634.333856,191.007988,14825.341844,26766277.924565,...,76.1,23.9,195.0,14.7,False,Acton-Boxborough,6000000.0,"$32,945,978","$86,677",380.1
4,Acton-Boxborough,Blanchard Memorial School,0,0,464.0,KG-06,16005.357328,289.306328,16294.663656,7481971.038995,...,91.4,8.6,79.3,14.3,False,Acton-Boxborough,6000000.0,"$32,945,978","$86,677",380.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,Worcester,Worcester East Middle,0,0,766.0,07-08,14603.898777,923.333657,15527.232434,11106589.551121,...,62.5,37.5,82.0,12.8,False,Worcester,3480000.0,"$151,535,395","$80,121","1,891.3"
1449,Worcester,Worcester Technical High,0,0,1426.0,09-12,16973.789244,812.745854,17786.535098,23741804.808678,...,49.0,51.0,176.8,10.9,False,Worcester,3480000.0,"$151,535,395","$80,121","1,891.3"
1450,Worthington,R. H. Conwell,1,1,74.0,PK-06,16583.928533,219.228195,16803.156728,1239095.860225,...,83.8,16.3,16.0,10.6,False,Worthington,3490000.0,"$376,966","$53,852",7.0
1451,Wrentham,Charles E Roderick,0,0,442.0,04-06,16436.024662,279.413743,16715.438405,7210757.95304,...,94.8,5.2,63.0,12.8,False,Wrentham,3500000.0,"$6,138,763","$76,543",80.2


In [1072]:
master_df.columns

Index(['distname', 'schoolname', 'flag_nerds', 'flag_f33', 'ncesenroll',
       'gradespan', 'pp_stloc_raw_MA', 'pp_fed_raw_MA', 'pp_total_raw_MA',
       'schoolstloc_raw_MA', 'schoolfed_raw_MA', 'schooltot_raw_MA',
       'distname_lower', 'schoolname_lower', 'School Name', 'School Code',
       'ELA_elem', 'MATH_elem', 'SCI', 'avg_score_elem', 'ELA_hs', 'MATH_hs',
       'avg_score_hs', 'avg_score', 'African American', 'Asian', 'Hispanic',
       'White', 'Native American', 'Native Hawaiian, Pacific Islander',
       'Multi-Race, Non-Hispanic', 'Males', 'Females', '# in Cohort',
       '% Graduated', '% Still in School', '% Non-Grad Completers',
       '% H.S. Equiv.', '% Dropped Out', '% Permanently Excluded',
       'First Language Not English #', 'First Language Not English %',
       'English Language Learner #', 'English Language Learner %',
       'Students With Disabilities #', 'Students With Disabilities %',
       'Low Income #', 'Low Income %', 'Free Lunch #', 'Free Lunch 

In [1073]:
master_df.to_csv("../output/master_df.csv", index=False)

## Step 1: Exploratory Data Analysis